In [16]:
# Import the necessary modules
import pandas as pd
import numpy as np
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

#training Modules
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression

#Evaluation Modules
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, classification_report



In [2]:
#reading the dataset files

train_df = pd.read_json("training_set.json")
test_df = pd.read_json("test_set.json")
# hashtag_corpus = pd.read_json("NTUSDFinCorpus/NTUSD_Fin_hashtag_v1.0.json")
# word_corpus = pd.read_json("NTUSDFinCorpus/NTUSD_Fin_word_v1.0.json")
emoji_corpus = pd.read_json("NTUSD_Fin_emoji_v1.0.json")

emoji_corpus

conditions = [
    (train_df['sentiment'] == 0) , 
    (train_df['sentiment'] <  0) ,
    (train_df['sentiment'] >  0)]
choices = ['neutral', 'bullish', 'bearish']
train_df['classes'] = np.select(conditions, choices, default='neutral')

# Pre-Processing

In [3]:
#placeholders 
rep = 0  #index for placeholders
p_mentions = [" @mentions ", " "]
p_cashtag = [" @cashtag ", " "]
p_url = [" @url ", " "]
stopwords = set(stopwords.words("english"))
qmark = " qmark "  
emark = " emark "

#### processing steps
- lowercase conversion
- replace mentions
- replace cashtag
- replace urls
- replace special unicode characters (&, > , < ,' )
- removing stopwords

yet to do
- removal of punctuations
- number processing 
- emoji

In [4]:
train_df['tweet'] = train_df['tweet'].str.lower()
train_df['tweet'] = train_df['tweet'].str.replace('([@][\w_-]+)', p_mentions[rep], case=False)
train_df['tweet'] = train_df['tweet'].str.replace('([$][a-z]+)', p_cashtag[rep], case=False)
train_df['tweet'] = train_df['tweet'].str.replace('http\S+|www.\S+', p_url[rep], case=False)
train_df['tweet'] = train_df['tweet'].str.replace('&amp', " & ", case=False)
train_df['tweet'] = train_df['tweet'].str.replace('&#39;', "'", case=False)
test_df['tweet'] = test_df['tweet'].str.replace('&gt;', " ", case=False)
test_df['tweet'] = test_df['tweet'].str.replace('&lt;', " ", case=False)
train_df['tweet'] = train_df['tweet'].str.replace('\?', qmark, case=False)
train_df['tweet'] = train_df['tweet'].str.replace('!', emark, case=False)
train_df['tweet'] = train_df['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))


preprocessing steps taken for test dataset is the same for training dataset

In [5]:
test_df['tweet'] = test_df['tweet'].str.lower()
test_df['tweet'] = test_df['tweet'].str.replace('([@][\w_-]+)', p_mentions[rep], case=False)
test_df['tweet'] = test_df['tweet'].str.replace('([$][a-z]+)', p_cashtag[rep], case=False)
test_df['tweet'] = test_df['tweet'].str.replace('http\S+|www.\S+',  p_url[rep], case=False)
test_df['tweet'] = test_df['tweet'].str.replace('&amp', " & ", case=False)
test_df['tweet'] = test_df['tweet'].str.replace('&#39;', "'", case=False)
test_df['tweet'] = test_df['tweet'].str.replace('&gt;', " ", case=False)
test_df['tweet'] = test_df['tweet'].str.replace('&lt;', " ", case=False)
test_df['tweet'] = test_df['tweet'].str.replace('\?', qmark, case=False)
test_df['tweet'] = test_df['tweet'].str.replace('!', emark, case=False)
test_df['tweet'] = test_df['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

In [6]:
for tweet in train_df.tweet:
    if "!" in tweet:
        print(tweet)

@gerberkawasaki stock hasn't moved much since first few weeks after split but still long term fan! $sbux
good morning! here's some of the most trending among #investors this morning. 
$vrx $aapl $outr $dsw $dwti $mjn $alr $plce $sppi $tif
$aapl $fb $amzn $pg $mcd $gs $c $ge $utx $slb $xom $googl $agn $wmt $qqq $iwm $gild $hon $jpm $dis nice day rally! https://t.co/usct2newzh
$twtr fck you bird! $fb is soaring. will buy it at first backtest of broken resistance. not here. very overbought. #tradeideas #stocks
rally friday! #new52weekhighs big and small $t $jnj $pm $wm $syy $cpb $twc $isrg $ultk and $fcty - up over 31%. have a fine weekend!
rally friday! #new52weekhighs big and small $t $jnj $pm $wm $syy $cpb $twc $isrg $ultk and $fcty - up over 31%. have a fine weekend!
@mobile_reach @anshublog $aapl is working with local financing agents to allow for the growing middle class in india to buy the iphone se!
@reutersbiz ahhh... the secret to curing #overbought conditions of the stock. good

In [7]:
train_df

,sentiment,snippet,target,tweet
0,-0.463,downgrade,$PCAR,downgrades $son $ari $gg $fltx $wmc $mfa $ivr ...
1,0.678,"[looking sexy this morning, break on volume]",$AMZN,$amzn looking sexy this morning...$600 break o...
2,0.377,still long term fan!,$SBUX,@gerberkawasaki stock hasn't moved much since ...
3,0.129,$TFM will have a way to go price wise to compe...,$KR,whole foods $wfm may feel price competition bu...
4,0.395,iPhone SE Could Be Doing Better Than Expected,$AAPL,apple's iphone se could be doing better than e...
5,0.458,Now up to 200 stocks making new 52 week highs,$TJX,now up to 200 stocks making new 52 week highs ...
6,0.288,There's no reason why $GOOGL can't get back to...,$GOOGL,there's no reason why $googl can't get back to...
7,-0.453,short,$CVEO,nice names on watch for tomorrow $tues long $8...
8,0.372,Pivotal sees 27% upside for Alphabet,$YHOO,pivotal sees 27% upside for alphabet https://t...
9,0.216,the most trending,$DWTI,good morning! here's some of the most trending...


# Training the Model (Linear Regression) 

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer( analyzer='word',
                      ngram_range=(1,3),
                      stop_words = 'english')

In [9]:
model = cv.fit_transform(list(train_df["tweet"]))

X = list(train_df["tweet"])
y = train_df["sentiment"]
X = cv.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.33, random_state=42 )

In [10]:
cv.fit(train_df["tweet"])
X_train = cv.transform(train_df["tweet"])
X_test = cv.transform(test_df["tweet"])
y_train = train_df["sentiment"]
y_test = test_df["sentiment"]

In [11]:
#linear regression model
from sklearn.linear_model import LinearRegression
log_model = LinearRegression(fit_intercept=True)
log_model = log_model.fit(X_train, y_train)
y_pred = log_model.predict(X_test)

# Evaluation

In [12]:
def assignClasses(data): 
    value = list()
    for i in data: 
        if i > 0:
            value.append("bullish")
        elif i < 0:
            value.append("bearish")
        else:
            value.append("neutral")
    
    return value 

In [13]:
n_y_test = assignClasses(y_test)
n_y_pred = assignClasses(y_pred)

In [19]:
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

print("MSE: ", mean_squared_error(y_test, y_pred))
print('\n')
print("F1 Macro Avg: ", f1_score(n_y_test, n_y_pred, average='macro'))
print("F1 Micro Avg: ", f1_score(n_y_test, n_y_pred, average='micro'))
print('\n')
print("Classification Report  \n", classification_report(n_y_test, n_y_pred))

MSE:  0.0912120874242


F1 Macro Avg:  0.490920106137
F1 Micro Avg:  0.758675078864


Classification Report  
              precision    recall  f1-score   support

    bearish       0.70      0.61      0.65       221
    bullish       0.78      0.87      0.82       401
    neutral       0.00      0.00      0.00        12

avg / total       0.74      0.76      0.75       634



D:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
